In [ ]:
import re
import pandas as pd
from pykospacing import Spacing

import torch
import rhinoMorph
from transformers import BertTokenizer, BertModel


# Rhino 객체 초기화
rn = rhinoMorph.startRhino()

In [13]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()                 
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

stopwords = get_stopwords()

# 형태소 분석 및 불용어 제거 함수
def morph_and_remove_stopwords(text, stopwords):
    text = preprocessing(text)
    morphs = rhinoMorph.onlyMorph_list(rn, text)
    meaningful_words = [word for word in morphs if word not in stopwords]
    return meaningful_words

In [14]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용
df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))

,feature,target,processed_feature
0,2032년 12월 samsung 주가가 어떻게 돼?,stock,"[2032, 12, SAMSU, NG, 주가, 어떻하, 게, 되]"
1,7월 7일 samsung 주가가 어떻게 돼?,stock,"[SAMSU, NG, 주가, 어떻하, 게, 되]"
2,5월 월말 samsung 주가가 어떻게 돼?,stock,"[말, SAMSU, NG, 주가, 어떻하, 게, 되]"
3,30일 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[30, 평일, 마지막, 주, SAMSU, NG, 주가, 어떻하, 게, 되]"
4,2월 금요일 samsung 주가가 어떻게 돼?,stock,"[금요일, SAMSUNG, 주가, 어떻하, 게, 되]"


In [16]:
# KoBERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

def text_to_vector(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():                                    # 모델에 입력하여 출력 벡터 얻기
        outputs = model(**inputs)
    cls_vector = outputs.last_hidden_state[0][0].numpy()     # [CLS] 토큰에 대한 벡터 추출
    return cls_vector

df['vector'] = df['processed_feature'].apply(text_to_vector) # 텍스트를 벡터로 변환하여 데이터프레임에 추가

                               feature target  \
0         2032년 12월 samsung 주가가 어떻게 돼?  stock   
1             7월 7일 samsung 주가가 어떻게 돼?  stock   
2             5월 월말 samsung 주가가 어떻게 돼?  stock   
3       30일 평일 마지막주 samsung 주가가 어떻게 돼?  stock   
4            2월 금요일 samsung 주가가 어떻게 돼?  stock   
...                                ...    ...   
140149                    오프라인 방법 궁금한데    FAQ   
140150                   오프라인 방법 좀 알려줘    FAQ   
140151                 오프라인 방법 이거 설명해줘    FAQ   
140152                오프라인 방법 상황 좀 보여줘    FAQ   
140153                 오프라인 방법 확인 좀 해줘    FAQ   

                                 processed_feature  \
0             [2032, 12, SAMSU, NG, 주가, 어떻하, 게, 되]   
1                       [SAMSU, NG, 주가, 어떻하, 게, 되]   
2                    [말, SAMSU, NG, 주가, 어떻하, 게, 되]   
3       [30, 평일, 마지막, 주, SAMSU, NG, 주가, 어떻하, 게, 되]   
4                    [금요일, SAMSUNG, 주가, 어떻하, 게, 되]   
...                                            ...   
140149                          [